In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [339]:
test = pd.read_csv("test1.csv")
df = pd.read_csv("train1.csv")
dev = pd.read_csv("dev1.csv")
#test.drop(test.columns[[13,14]], axis=1, inplace=True)
#df.drop(df.columns[[13,14]], axis=1, inplace=True)

In [340]:
x_train = df.iloc[:,4:-1]
y_train = df.label
x_dev = dev.iloc[:,4:-1]
y_dev = dev.label

In [333]:
df.shape

(28846, 9)

In [334]:
dev.shape

(22036, 9)

In [338]:
len(dev[dev.label ==0])

16036

In [290]:
df.iloc[:,4:-1]

,aa,ra,jc,pa
0,2.471649,0.531746,0.428571,96
1,2.976054,0.668651,0.466667,112
2,2.899858,0.628968,0.700000,72
3,2.471649,0.531746,0.600000,64
4,2.976054,0.668651,0.466667,112
...,...,...,...,...
32067,0.000000,0.000000,0.000000,50
32068,0.000000,0.000000,0.000000,70
32069,0.000000,0.000000,0.000000,20
32070,0.000000,0.000000,0.000000,50


In [286]:
test.iloc[:,4:]

,aa,ra,jc,pa
0,0.000000,0.000000,0.000000,56
1,0.000000,0.000000,0.000000,24
2,0.000000,0.000000,0.000000,496
3,1.242670,0.400000,0.080000,72
4,0.000000,0.000000,0.000000,391
...,...,...,...,...
1995,0.000000,0.000000,0.000000,2
1996,0.000000,0.000000,0.000000,2
1997,0.000000,0.000000,0.000000,116
1998,1.766048,0.424368,0.053333,370


In [280]:
y = [True if i < df.shape[0]/2 else False for i in range(df.shape[0])]
x = df.iloc[:,4:13]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4)

In [281]:
collections.Counter(y)

Counter({True: 16036, False: 16036})

In [341]:
#from sklearn.preprocessing import StandardScaler
#normal = StandardScaler()
#x_train = normal.fit_transform(x_train)
#x_test = normal.transform(x_test)
#l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'
LR = LogisticRegression(n_jobs = 4, penalty = 'l1', solver = 'liblinear').fit(df.iloc[:,4:-1], df.label)
LRT = LR.predict(dev.iloc[:,4:-1])

print(metrics.classification_report(dev.label, LRT))


              precision    recall  f1-score   support

           0       0.86      0.97      0.91     16036
           1       0.87      0.60      0.71      6000

    accuracy                           0.86     22036
   macro avg       0.86      0.78      0.81     22036
weighted avg       0.86      0.86      0.86     22036



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [342]:
probs = LR.predict_proba(dev.iloc[:,4:-1])

In [343]:
metrics.roc_auc_score(dev.label, probs[:, 1])

0.8220712615365426

In [344]:
import collections
collections.Counter(LR.predict(test.iloc[:,4:]))

Counter({0: 1439, 1: 561})

In [345]:
results = LR.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.07217064, 0.07062956, 0.07106674, ..., 0.07528835, 1.        ,
       0.07030331])

In [278]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('logReg.csv', header = True, index= False)

### Handmade gridsearch

In [331]:
import itertools as it
from sklearn.metrics import accuracy_score
LR = LogisticRegression
LR_param = {'penalty':['l1', 'l2', 'elasticnet'],
            'C':[1.0,0.5,0.1,0.01], 'max_iter':[200],
           'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
models = [('Logistic Regression',LR,LR_param)]

result = []
best = {}
for name, m, params in models:
    keys, values = zip(*params.items())
    permutations = [dict(zip(keys, v)) for v in it.product(*values)]
    
    for p in permutations:
        try:
            mod = m(**p)
            mod.fit(x_train, y_train);
            y_pred = mod.predict(x_dev)
            Acc_scored = accuracy_score(y_dev, y_pred)
            cnt = collections.Counter(mod.predict(test.iloc[:,4:]))
            best[str(p)] = (Acc_scored, cnt)
            #print('Model: ',name, ' HyperParams: ',p,' Accuracy: ',Acc_scored)
        except:
            pass#print(p)

    #best_params = max(best, key = lambda k:best[k][0])
    #result.append((name, best_params, best[best_params][0], best[best_params][1]))  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did 

In [332]:
[print(k,v) for k,v in best.items()]

{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'} (0.8648575058994373, Counter({0: 1439, 1: 561}))
{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'saga'} (0.39117807224541656, Counter({1: 1636, 0: 364}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'liblinear'} (0.8648575058994373, Counter({0: 1439, 1: 561}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'saga'} (0.39117807224541656, Counter({1: 1636, 0: 364}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'liblinear'} (0.8648121256126339, Counter({0: 1439, 1: 561}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'saga'} (0.39117807224541656, Counter({1: 1636, 0: 364}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'liblinear'} (0.8643129424577963, Counter({0: 1442, 1: 558}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'saga'} (0.39117807224541656, Counter({1: 1636, 0: 364}))
{'penalty': 'l2', 'C': 1.0, 'max_iter': 200, 'solver': 'newton-cg'} (0.8648121

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### LGBM

In [323]:
import lightgbm as lgbm

train_data = lgbm.Dataset(x_train, pd.Series(y_train))
test_data = lgbm.Dataset(x_dev, pd.Series(y_dev))

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'num_threads' : 2,
    'seed' : 76
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=25)

[LightGBM] [Info] Number of positive: 12810, number of negative: 16036
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000111 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1018
[LightGBM] [Info] Number of data points in the train set: 28846, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444082 -> initscore=-0.224610
[LightGBM] [Info] Start training from score -0.224610
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.780831
Training until validation scores don't improve for 25 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.780657
[3]	valid_0's auc: 0.823515
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 0.823689
[LightGBM] [Warning] No further splits wit

In [326]:
predict

array([0.49348902, 0.49348902, 0.49348902, ..., 0.49348902, 0.49348902,
       0.49348902])

In [328]:
predict= model.predict(test.iloc[:,4:])
collections.Counter(["pos" if i>0.5 else "neg" for i in predict])

Counter({'neg': 1439, 'pos': 561})

### Support Vector Machine (SVM)

In [325]:
SVM = LinearSVC(max_iter=10000)
#SVM_predict = SVM.predict(x_test)
clf = CalibratedClassifierCV(SVM) 
clf.fit(x_train, y_train)
clf_predict = clf.predict(x_dev)


print(metrics.classification_report(y_dev, clf_predict))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.82      0.99      0.90     16036
           1       0.97      0.43      0.59      6000

    accuracy                           0.84     22036
   macro avg       0.89      0.71      0.75     22036
weighted avg       0.86      0.84      0.82     22036



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [58]:
probs = clf.predict_proba(x_test)

In [59]:
probs

array([[0.67533204, 0.32466796],
       [0.34029695, 0.65970305],
       [0.35430718, 0.64569282],
       ...,
       [0.80227792, 0.19772208],
       [0.67984221, 0.32015779],
       [0.67347837, 0.32652163]])

In [330]:
metrics.roc_auc_score(y_dev, probs[:, 1])

0.8220605824395111

In [61]:
collections.Counter(clf.predict(test.iloc[:,4:]))

Counter({'neg': 1465, 'pos': 535})

In [42]:
collections.Counter(y_test)

Counter({'neg': 4029, 'pos': 3989})

In [20]:
results = clf.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.32588784, 0.11201567, 0.32562297, ..., 0.32601893, 0.65587601,
       0.11203549])

In [160]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': predict})
output.to_csv('lightGBM.csv', header = True, index= False)